# Multi-Layered Perceptron via Grid Search CV # 

In [59]:
#import libraries
import numpy as np
import pandas
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report
import time
from sklearn.preprocessing import StandardScaler, LabelEncoder

### Red Wines Dataset ###

In [60]:
#import data
label_quality = LabelEncoder()
data_red = pandas.read_csv('winequality-red.csv', delimiter=';')
data_red['quality'] = label_quality.fit_transform(data_red['quality'])
y = data_red['quality']
X = data_red[data_red.columns[:-1]]

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [61]:
# Normalize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

N_test_red = len(X_test)

In [62]:
#Neural Network to generate predictions
def predNN_2(X_train, y_train, wine_color = 'red'):
    clf = GridSearchCV(estimator=MLPClassifier(max_iter=50000, random_state=42),
          param_grid={'early_stopping' : [True, False],
                      'hidden_layer_sizes': [100, 200, 300, 400, 500],
                      'activation': ['relu', 'tanh', 'logistic'],
                      'learning_rate': ['constant', 'invscaling', 'adaptive'],
                      'learning_rate_init': [.0001, .001, .01, .1]},
                      n_jobs = -1, verbose = 4)
    
    # Note (put in report): we picked the default 'adam' solver since, according to the scikit-learn documentation for MLPClassifier, 
    # it "works pretty well on relatively large datasets (with thousands of training samples or more) 
    # in terms of both training time and validation score." Note that 'lgbfs' is very slow!!
    
    # Note: GridSearchCV by default performs 5-fold CV. 
    
    print("Fitting")
    t0 = time.time()
    clf.fit(X_train, y_train)
    t1 = time.time()
    print('Grid search CV time for', wine_color, 'wines took', t1 - t0, 'seconds')
    print("Optimized parameters:", clf.best_params_)
    print("Weighted validation score:", clf.best_score_)
    return clf

In [63]:
#Calling Neural Network for Predictions
classifier = predNN_2(X_train, y_train, 'red')
preds = classifier.predict(X_test)

Fitting
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Grid search CV time for red wines took 2361.737888097763 seconds
Optimized parameters: {'activation': 'logistic', 'early_stopping': False, 'hidden_layer_sizes': 500, 'learning_rate': 'constant', 'learning_rate_init': 0.1}
Weighted validation score: 0.6254810049019608


In [64]:
# write cv results (such as mean fit times for each hyperparam configuration) to file
with open('fcnn_gridcv_red_adam.txt', 'w') as file:
    file.write(str(classifier.cv_results_))

In [65]:
activation = classifier.best_params_['activation']
hidden_layer_sizes = classifier.best_params_['hidden_layer_sizes']
learning_rate = classifier.best_params_['learning_rate']
learning_rate_init = classifier.best_params_['learning_rate_init']
early_stopping = classifier.best_params_['early_stopping']

classifier = MLPClassifier(early_stopping=early_stopping, activation=activation, hidden_layer_sizes=hidden_layer_sizes, max_iter=50000, learning_rate=learning_rate, learning_rate_init=learning_rate_init)
t0 = time.time()
classifier.fit(X_train, y_train)
t1 = time.time()
print('Training time time for red wines took', t1 - t0, 'seconds')

Training time time for red wines took 3.13327693939209 seconds


In [66]:
#training accuracy
train_accNN = accuracy_score(y_train, classifier.predict(X_train)) 
train_accNN

0.9984362783424551

In [67]:
#test accuracy
test_accNN_red = accuracy_score(y_test, classifier.predict(X_test)) 
test_accNN_red

0.653125

### White Wines Dataset ###

In [68]:
#import data
label_quality = LabelEncoder()
data_white = pandas.read_csv('winequality-white.csv', delimiter=';')
data_white['quality'] = label_quality.fit_transform(data_white['quality'])
y = data_white['quality']
X = data_white[data_white.columns[:-1]]

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [69]:
# Normalize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

N_test_white = len(X_test)

In [70]:
#Calling Neural Network for Predictions
classifier = predNN_2(X_train, y_train, 'white')
preds = classifier.predict(X_test)

Fitting
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Grid search CV time for white wines took 17003.405514001846 seconds
Optimized parameters: {'activation': 'relu', 'early_stopping': False, 'hidden_layer_sizes': 500, 'learning_rate': 'constant', 'learning_rate_init': 0.001}
Weighted validation score: 0.6135774233064873


In [71]:
# write cv results (such as mean fit times for each hyperparam configuration) to file
with open('fcnn_gridcv_white_adam.txt', 'w') as file:
    file.write(str(classifier.cv_results_))

In [72]:
activation = classifier.best_params_['activation']
hidden_layer_sizes = classifier.best_params_['hidden_layer_sizes']
learning_rate = classifier.best_params_['learning_rate']
learning_rate_init = classifier.best_params_['learning_rate_init']
early_stopping = classifier.best_params_['early_stopping']

classifier = MLPClassifier(early_stopping=early_stopping, activation=activation, hidden_layer_sizes=hidden_layer_sizes, max_iter=50000, learning_rate=learning_rate, learning_rate_init=learning_rate_init)
t0 = time.time()
classifier.fit(X_train, y_train)
t1 = time.time()
print('Training time time for white wines took', t1 - t0, 'seconds')

Training time time for white wines took 46.954169034957886 seconds


In [73]:
#training accuracy
train_accNN = accuracy_score(y_train, classifier.predict(X_train)) 
train_accNN

0.9834099030117407

In [74]:
#test accuracy
test_accNN_white = accuracy_score(y_test, classifier.predict(X_test)) 
test_accNN_white

0.6479591836734694

### Weighted Accuracy ###

In [75]:
(N_test_red * test_accNN_red + N_test_white * test_accNN_white) / (N_test_white + N_test_red)

0.6492307692307693